In [1]:
import numpy as np 
import pandas as pd 

from preprocessing import *
from model import *
from utils import *

In [2]:
cat_data = pd.read_csv('cat_data_del_down_pre.csv', encoding='utf8')

cat_data

,id,post_message,label,post_message_preproced
0,448,"Hoá đơn tiền điện tăng ""sốc"": Kẽ hở trong việc...",0,hoá_đơn tiền điện sốc kẽ hở ghi điện việt_nam ...
1,7,Chủ đầu tư tại Hà Nội tung sản phẩm có mức giá...,0,chủ đầu_tư hà_nội tung sản_phẩm giá tỷ đồng đó...
2,1278,Công an huyện Kbang ra quyết định tạm giữ hình...,0,công_an huyện kbang quyết_định tạm hình_sự phạ...
3,2745,Quá thương tâm\nPhó Chủ tịch UBND quận Bình Tâ...,0,thương_tâm phó chủ_tịch ubnd bình_tân tphcm nạ...
4,2009,Công ty TNHH Thương mại Thủy sản Vĩnh Long đã ...,0,công_ty tnh thương_mại thuỷ_sản vĩnh_long lợi_...
...,...,...,...,...
1451,4336,"TQ 1980 có GDP 190 tỷ USD, thua VN bây giờ. Sa...",1,tq gdp tỷ usd thua vn gdp tỷ usd lịch_sử ca gi...
1452,4343,Virus Corona là vũ khí sinh học? Hiện nay có n...,1,virus corona vũ_khí_sinh_học bằng_chứng virus ...
1453,4364,Tin từ Phó Chủ Tịch UBND TPHCM Lê Thanh Liêm.\...,1,phó chủ_tịch ubnd tphcm lê_thanh_liêm tp hồ_ch...
1454,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,1,cảnh_tỉnh thanh_niên dân tổtốc độ bàn_thờ emoj...


In [6]:
num_data = pd.read_csv('num_data_del_down.csv', encoding='utf8')

num_data

,id,num_like_post,num_comment_post,num_share_post,label
0,4341,400,15,79,0
1,1035,40000,1400,2600,0
2,2896,13,1,0,0
3,3941,6,0,0,0
4,1805,6,1,0,0
...,...,...,...,...,...
1451,4336,25,5,0,1
1452,4343,4800,436,3100,1
1453,4364,13,5,0,1
1454,4370,3,1,0,1


In [4]:
X_cat = cat_data.post_message_preproced.values

pipeline, X_tfidf = text_tfidf(X_cat)

print(X_tfidf.shape)

(1456, 11581)


In [8]:
cols = ['num_like_post', 'num_comment_post', 'num_share_post']
X_num, y = num_data[cols].values, num_data.label.values.reshape(-1,1)

print(X.shape)

(1456, 3)


In [9]:
data = np.hstack((X_tfidf, X_num))

print(data.shape)

(1456, 11584)


In [10]:
options = [0,1,2,3,4,5]
names = ['KNN','Gaussian', 'SVM', 'Randomforest', 'Logistc', 'Stack']

for (option, name) in zip(options, names):
    print(f'{name} model:')
    _ = models(data, y, option=option)
    print('*'*170)

KNN model:
Report of train set: 
Time to predict train set: 0.6629s
              precision    recall  f1-score   support

           0       0.77      0.81      0.79       637
           1       0.76      0.71      0.73       527

    accuracy                           0.76      1164
   macro avg       0.76      0.76      0.76      1164
weighted avg       0.76      0.76      0.76      1164


Report of test set:
Time to predict test set 0.9668s
              precision    recall  f1-score   support

           0       0.73      0.75      0.74       163
           1       0.68      0.65      0.66       129

    accuracy                           0.71       292
   macro avg       0.70      0.70      0.70       292
weighted avg       0.71      0.71      0.71       292

**************************************************************************************************************************************************************************
Gaussian model:
Report of train set: 
Time to predict

In [11]:
import tensorflow as tf 
import keras
from keras import layers

In [20]:
model = keras.Sequential()
model.add(layers.Input(shape=data.shape))
model.add(layers.Dense(units=100, activation='relu'))
model.add(layers.Dense(units=200, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1456, 100)         1158500   
_________________________________________________________________
dense_7 (Dense)              (None, 1456, 200)         20200     
_________________________________________________________________
dense_8 (Dense)              (None, 1456, 1)           201       
Total params: 1,178,901
Trainable params: 1,178,901
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer="Adam", loss="mse", metrics=["mae", "acc"])
model.fit(data, y, epochs=100, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/100
37/37 [==============================] - 0s 11ms/step - loss: 0.2504 - mae: 0.4136 - acc: 0.6976 - val_loss: 0.4336 - val_mae: 0.5688 - val_acc: 0.2705
Epoch 2/100
37/37 [==============================] - 0s 8ms/step - loss: 0.2111 - mae: 0.3431 - acc: 0.7242 - val_loss: 0.6784 - val_mae: 0.8060 - val_acc: 0.0171
Epoch 3/100
37/37 [==============================] - 0s 9ms/step - loss: 0.2075 - mae: 0.3229 - acc: 0.7337 - val_loss: 0.5468 - val_mae: 0.6621 - val_acc: 0.2877
Epoch 4/100
37/37 [==============================] - 0s 8ms/step - loss: 0.2403 - mae: 0.3027 - acc: 0.7156 - val_loss: 0.7703 - val_mae: 0.8586 - val_acc: 0.0479
Epoch 5/100
37/37 [==============================] - 0s 8ms/step - loss: 0.2051 - mae: 0.2781 - acc: 0.7775 - val_loss: 0.8423 - val_mae: 0.8783 - val_acc: 0.0822
Epoch 6/100
37/37 [==============================] - 0s 8ms/step - loss: 0.2293 - mae: 0.2664 - acc: 0.7646 - val_loss: 0.6595 - val_mae: 0.7302 - val_acc: 0.2877
Epoch 7/100
37/37 [==